In [66]:
import pandas as pd
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity

# Data exploration

In [182]:
data_path = 'data/spotify.sqlite'
conn = sqlite3.connect(data_path)
conn.text_factory = lambda b: b.decode(errors = 'ignore')


In [183]:
cursor = conn.cursor()

query = '''
    SELECT
        af.id,
        af.acousticness,
        af.danceability, 
        af.energy,
        af.instrumentalness,
        af.key,
        af.liveness,
        af.loudness, 
        af.mode,
        af.speechiness,
        af.tempo,
        af.time_signature,
        af.valence
    FROM audio_features af
'''
cursor.execute(query)
af = cursor.fetchall()
# af = pd.read_sql(query, conn).sort_values('id')

In [117]:
ids = tuple(af_features.id.values)

In [122]:
query = f'SELECT * FROM tracks WHERE id IN {ids}'

tracks = pd.read_sql(query, conn).sort_values('id').loc[:, ['id', 'name']]

In [147]:
query = f'SELECT * FROM r_track_artist WHERE track_id in {ids}'
track_artist = pd.read_sql(query, conn).rename(columns={'artist_id': 'id'})

In [134]:
artist_ids = tuple(track_artist.artist_id.values)

In [142]:
query = f'SELECT * FROM artists WHERE id in {artist_ids}'
artists = pd.read_sql(query, conn).loc[:, ['id', 'name']].set_index('id')

In [172]:
# merge track_artist and artists
track_artist_name = track_artist.join(artists, on='id', how='left')

In [173]:
track_artist_name = track_artist_name.groupby('track_id').agg({'name': '; '.join})

In [174]:
df = af.join(tracks.set_index('id'), on='id', how='left').join(track_artist_name, on='id', how='left', lsuffix='_track', rsuffix='_artist')

In [180]:
df.head(50)

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,name_track,name_artist
72,02FzJbHtqElixxCmrpSCUa,0.996000,0.474,0.2390,0.186000,9,0.1950,-9.712000,1,0.0289,78.783997,3,0.3660,Arkansas Blues,Mamie Smith & Her Jazz Hounds
23,0B2ZuTLZnWQ6gz8RlNwBu1,0.092300,0.759,0.8250,0.000570,8,0.2400,-4.289000,1,0.2200,94.857002,4,0.4290,On Fire,Lloyd Banks
24,0EZMXJMWf0tLKRWwCiA6Sx,0.055000,0.905,0.8500,0.000000,4,0.0903,-2.795000,0,0.3310,99.375000,4,0.4980,Let's Get High,Dr. Dre; Hittman; Ms. Roq; Kurupt
13,0HNAPf0cLMkVQfwPl74kF3,0.409000,0.646,0.7960,0.000000,2,0.1070,-6.152000,1,0.3660,83.093002,4,0.6760,Ass Like That,Eminem
91,0KGiP9EW1xtojDHsTGARL5,0.996000,0.468,0.5330,0.870000,1,0.1790,-11.002000,1,0.0452,136.572998,4,0.7280,El Africano - Remasterizado,Francisco Canaro
87,0TIfXVGqyiOTuWkuxEzQzc,0.000012,0.568,0.7290,0.912000,2,0.0613,-8.293000,1,0.0312,129.987000,4,0.1750,True House Music - Original Massive Mix,Oscar Velazquez
85,0cS0A1fUEUd1EW3FcF8AEI,0.991000,0.598,0.2240,0.000522,5,0.3790,-12.628000,0,0.0936,149.975998,4,0.6340,Keep A Song In Your Soul,Mamie Smith
86,0hbkKFIJm7Z05H8Zl9w30f,0.643000,0.852,0.5170,0.026400,5,0.0809,-7.261000,0,0.0534,86.889000,4,0.9500,I Put A Spell On You,Screamin' Jay Hawkins
78,0lqEx4vktZP1y9hnwfF27Y,0.995000,0.482,0.2290,0.000060,7,0.5490,-12.619000,1,0.0812,77.232002,4,0.4610,Kansas City Man Blues - 78rpm Version,Mamie Smith; The Harlem Trio
76,0sj3QeaCpacCfxL4c5TQBl,0.995000,0.534,0.1910,0.098200,7,0.8050,-12.889000,0,0.0508,70.058998,4,0.6050,Mean Man,Mamie Smith & Her Jazz Hounds


In [176]:
# select a song to input

query = "SELECT * FROM tracks WHERE name = 'Girlfriend'"
input = pd.read_sql(query, conn)

In [177]:
input

,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable
0,0VtrSyLoPpOIMlzfVPfbY7,1,217000,0,0VtrSyLoPpOIMlzfVPfbY7,Girlfriend,https://p.scdn.co/mp3-preview/1197cbe40fbfcd0d...,5,0,NaN
1,7wGOjwxFj8wbUwcu9rL47a,2,245253,0,7wGOjwxFj8wbUwcu9rL47a,Girlfriend,https://p.scdn.co/mp3-preview/b8a3d4b096b5ae09...,11,9,NaN
2,2uaTfYH3jfZ808xyeFif12,1,190240,0,2uaTfYH3jfZ808xyeFif12,Girlfriend,,9,17,NaN
3,6JqRzMVuX8sO5huQpG1vbi,1,171320,1,6JqRzMVuX8sO5huQpG1vbi,Girlfriend,https://p.scdn.co/mp3-preview/08a3942bf057ebbc...,5,45,NaN
4,63LozqLQXoviurS1xPV5FX,1,239746,0,63LozqLQXoviurS1xPV5FX,Girlfriend,https://p.scdn.co/mp3-preview/0558e52a24f1e962...,5,14,NaN
...,...,...,...,...,...,...,...,...,...,...
236,0hKR3GDXdzXQcXCkgFUNFa,1,72493,0,0hKR3GDXdzXQcXCkgFUNFa,Girlfriend,https://p.scdn.co/mp3-preview/cfbe28ffe31e8984...,10,3,NaN
237,0MsbKzHKuXEVUitS7e9nWa,1,187693,0,0MsbKzHKuXEVUitS7e9nWa,Girlfriend,,5,2,NaN
238,0mx99jEyxr83HLJ0y0JfKk,1,226000,0,0mx99jEyxr83HLJ0y0JfKk,Girlfriend,https://p.scdn.co/mp3-preview/594a75cdeb9882fa...,2,9,NaN
239,0nMEBbbohDmypdvpIKYqdw,1,165120,0,0nMEBbbohDmypdvpIKYqdw,Girlfriend,https://p.scdn.co/mp3-preview/9a6703d32b4985da...,3,4,NaN


In [92]:
# for index, row in af1.iterrows():
cos_sim = cosine_similarity(af1.iloc[:, 1:], af2.iloc[:, 1:])
af1['cos_sim'] = cos_sim
    # break

/tmp/ipykernel_22913/400334335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  af1['cos_sim'] = cos_sim


In [93]:
af1.sort_values('cos_sim')

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,cos_sim
51,2PxYJDohsBXaBLByHBITp5,0.955,0.366,0.0819,0.8880,5,0.0816,-18.674999,1,0.0437,49.112000,4,0.1210,0.967937
59,3jmUW1tGOpScXF5c0zXJOh,0.952,0.479,0.1790,0.9360,2,0.2080,-18.337000,1,0.0315,68.671997,3,0.2010,0.987620
47,5U6viBMb17ayuEad6Inpgg,0.980,0.462,0.1740,0.8720,3,0.1160,-18.377001,1,0.0365,69.302002,4,0.2120,0.988262
45,3WEkHcJisp1XyPPNMWKAi0,0.920,0.153,0.0186,0.3760,2,0.1070,-21.146000,1,0.0399,82.152000,4,0.1670,0.988622
94,6vZWpFl8wRQxQ69E9FoqZQ,0.996,0.462,0.2260,0.9480,3,0.1660,-16.773001,1,0.0642,66.658997,4,0.2200,0.989993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,5Ccyefda1pTeRCAg2kUIbX,0.983,0.429,0.2920,0.0700,6,0.3380,-12.436000,1,0.0315,95.543999,3,0.0734,0.999606
97,5bx9VS0wfBQfVYk0ujrBoB,0.996,0.616,0.3340,0.9460,8,0.2030,-17.108999,1,0.0473,130.572006,4,0.5610,0.999622
64,1DfwOz5apF0w2d8hccSkhH,0.996,0.575,0.2470,0.0684,9,0.3620,-13.284000,0,0.0400,130.804001,4,0.8410,0.999811
18,1ktyqk3vVMB3zg17NjttnX,0.847,0.639,0.6210,0.0208,8,0.3430,-9.981000,0,0.4030,92.582001,4,0.6000,0.999829


In [82]:
af1.iloc[:, 1:]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.1800,0.893,0.514,0.000000,11,0.0596,-5.080000,1,0.2830,95.848000,4,0.787
1,0.2720,0.520,0.847,0.000000,9,0.3250,-5.300000,1,0.4270,177.371002,4,0.799
2,0.0783,0.918,0.586,0.000000,1,0.1450,-2.890000,1,0.1330,95.516998,4,0.779
3,0.5840,0.877,0.681,0.000000,1,0.1190,-6.277000,0,0.2590,94.834999,4,0.839
4,0.1700,0.814,0.781,0.000518,11,0.0520,-3.330000,1,0.2330,93.445000,4,0.536
...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.9960,0.462,0.226,0.948000,3,0.1660,-16.773001,1,0.0642,66.658997,4,0.220
95,0.9950,0.472,0.223,0.980000,2,0.2990,-16.287001,1,0.0810,127.075996,4,0.372
96,0.9960,0.369,0.285,0.938000,7,0.1930,-16.523001,1,0.0528,129.557999,5,0.635
97,0.9960,0.616,0.334,0.946000,8,0.2030,-17.108999,1,0.0473,130.572006,4,0.561


In [90]:
af2.iloc[:, 1:]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
99,0.996,0.477,0.319,0.972,11,0.148,-15.328,0,0.0542,134.102997,4,0.61
